# Case Study 2: Deployment

In the previous notebook we were able to compute for all existing employees an attrition risk score. In the simplest case, the analysis can stop there and your final result will consist in a **list of employees sorted by decreasing score values**   (possibly limited to the **top-K** entries).

However, we can go further. Indeed, our model is supposed to have captured some important relationships between the various factors. This means that, if we can identify some variables on which we can act to modify the situation, we might be able to build some strategies to keep employees from leaving.


## Learnt model as a simulator

### Categorizing the variables

The first step will consist in identifying the variables which can be acted upon. We will make at least 3 categories:

* actionable variables;
* non-actionable variables that we know are independant of the actionable variables;
* non-actionable variables which are expected to be influenced by changes in the actionable variables (and which also have an influence on the model output, the attrition).

The last category is tricky because those variables might have a high influence on the model output (see features importance in previous notebooks), so we must be able to update them when changing the actionable values in order to predict an accurate output with the model we have trained.

For instance, it is likely that changing the average monthly work hour (our actionable variable) will have an impact on satisfaction level (our non-actionable dependant variable). Moreover, the model have learned a strong link between this satisfaction level and employee attrition.  If we were to use the predictive model for simulating the effect of changing the average monthly work hours, and did that without updating the satisfaction level, the prediction would degrade significantly.

There are two ways of dealing with this kind of variables:

* either build a sub-model able to infer them from the other (actionable or independant) variables, so that a sensible  value can be given as input to the attrition prediction model;

* or train a new attrition model where those variables have been removed (leading potentially to a loss of predictive performance).


Note: the way we have defined the 3 kinds of variables and their relations in the model is somewhat unprecise. There is a better way to model these effects, using a graphical model of the variables' probabilistic dependancies, and Bayesian methods. These are unfortunately out of the scope of this formation.

Let's have a look at our models and dataset:

In [ ]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../python")

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import joblib

models, df_attr = joblib.load("./model_data.pkl")

In [ ]:
df_attr.sort_values("attr_score", ascending=False, inplace=True)
df_attr

In [ ]:
from sklearn.metrics import average_precision_score

In [ ]:
print("AP:", average_precision_score(df_attr.has_left, df_attr.attr_score))

Let's compute the mean importance of the variables over all models. For this we evaluate the importance on the test set of each model of the cross validation:

In [ ]:
from sklearn.inspection import permutation_importance


def mean_importances(models, X, y, fold_index):
    importances = []
    for i, model in enumerate(models):
        mask = fold_index == i
        Xi = X[mask]
        yi = y[mask]
        res = permutation_importance(model, Xi, yi)
        importances.append(res["importances_mean"])
    return pd.Series(
        np.mean(importances, 0),
        index=pd.Index(X.columns, name="features"),
        name="importance",
    )


exclude_columns = ["Emp ID", "has_left", "attr_score", "fold_index"]
_ = (
    mean_importances(
        models,
        df_attr.drop(exclude_columns, axis=1),
        df_attr.has_left.values,
        fold_index=df_attr.fold_index,
    )
    .sort_values()
    .plot.barh()
)

Actionable variables are:

* `number_project`
* `average_monthly_hours`
* `salary`
* `promotion_last_5years`

### Non-actionable variables correlated with actionable variables

Among the important variables that are not directly actionable we have:

* `satisfaction_level`
* `last_evaluation`

Those two variables are probably linked to the actionable variables and are expected to change if the actionable variables are modified.

Let's check how they are linked:

In [ ]:
_ = sns.heatmap(
    df_attr.drop(exclude_columns + ["dept", "salary"], axis=1).corr(),
    vmin=-1,
    vmax=1,
    cmap="coolwarm",
    annot=True,
    fmt="0.2f",
)

We see that:

* the `satisfaction_level` variable is negatively correlated to the number of projects
* the `last_evaluation` variable is positively correlated to both actionable variables.

There are basically two ways to handle correlated non-actionable variables:

* model them in order to impute their values when the actionnable variables are modified;
* remove them and check that the overall performance is not degraded too much.

The first option should be considered if there is a known model for the variables. Since that is not the case here, we will remove those variables, and create a new attrition model. The hope is that the chosen estimator will be flexible enough to directly learn the relationship between the remaining features and the attrition.

Here we will remove:

* `satisfaction_level`
* `last_evaluation`

### Retraining the models

We shall reuse the existing model and retrain "manually" the models for each fold:

In [ ]:
exclude_columns = [
    "Emp ID",
    "has_left",
    "attr_score",
    "fold_index",
    "satisfaction_level",
    "last_evaluation",
]
X = df_attr.drop(exclude_columns, axis=1)
y = df_attr.has_left.values

In [ ]:
from sklearn.base import clone

model_gb_cv = models[0]  # Retrieve the initial model object
models_new = []
fold_index = np.zeros_like(y)
scores = np.zeros_like(y, dtype="float")

for i in range(len(models)):
    mask = df_attr.fold_index == i
    itest = np.where(mask)[0]
    itrain = np.where(~mask)[0]
    model = clone(model_gb_cv)
    model.fit(X.iloc[itrain], y[itrain])
    scores[itest] = model.predict_proba(X.iloc[itest]).T[1]
    fold_index[itest] = i
    models_new.append(model)

Note that the `numpy.where` function, used with a boolean array as unique argument returns a tuple containing the indexes of the `True` elements, with one index array per dimension.

Since in the case above, we have only one dimension, the tuple has only 1 index array.

In [ ]:
print(np.where([True, False, False, True, False, True]))
print(np.where([[True, False], [False, True], [False, True]]))

Let's evaluate the model quality:

In [ ]:
from sklearn.metrics import average_precision_score

print(average_precision_score(y, scores))
sns.kdeplot(x=scores, hue=y)
_ = plt.xlabel("attrition score")

Even though we removed the most important variables, we still have a model with a high score.

We can check the new feature importances:

In [ ]:
_ = mean_importances(models_new, X, y, df_attr.fold_index).sort_values().plot.barh()

So we have a model which has a degraded (as expected) but still exhibits a high performance score. The ordering of the features has also moved a bit.

Depending on the model type, when some variables are correlated (in the broad sense), it is possible that some important features rank low because another features already captures most of the same information.

This will be especially the case with gradient boosting based models which work incrementally.

In [ ]:
df_attr_new = df_attr.assign(attr_score=scores)

emp1 = (
    df_attr_new.query("has_left == 0")
    .sort_values("attr_score", ascending=False)
    .iloc[0]
)
emp1

So the next step will be to see how we can modify the values of the actionable variables for a single employee to lower the attrition score and make it pass under the 0.5 level for instance.

### Perform simulations

Let's build a function which will allow us to create modified versions of the original sample:

In [ ]:
from itertools import product


def modify_grid(entry, grid):
    # Compute all combinations of modified variables
    df_grid = pd.DataFrame(product(*grid.values()), columns=grid.keys())

    # Create a dataframe with len(df_grid) copies of the initial row
    df_modified = entry.to_frame().T.iloc[np.zeros(len(df_grid), dtype="int")]

    # Replace the modified values
    for c in df_grid.columns:
        df_modified[c] = df_grid[c].values

    return df_modified

This function takes an entry (a data frame row) and a grid of parameters (similar to the one we use for the grid search for instance).

The function returns a data frame where each row is built from the original entry, and the columns whose names correspond to the grid keys have been replaced with all the combinations of the grid values.


In [ ]:
import matplotlib.pyplot as plt

feature = "number_project"
grid_values = np.arange(1, 10)

grid = {feature: grid_values}
df_modified = modify_grid(emp1, grid)
df_modified

We can now simulate the attrition score change:

In [ ]:
model = models_new[emp1.fold_index]
scores_new = model.predict_proba(df_modified.drop(exclude_columns, axis=1)).T[1]

plt.plot(grid_values, scores_new, "o-", label="predicted score")
plt.plot([emp1[feature]] * 2, [0, 1], "--", color="gray", label="current value")
plt.xlabel(feature)
plt.ylabel("Attrition score")
_ = plt.legend()

In [ ]:
feature = "average_monthly_hours"
grid_values = np.linspace(100, 310, 21)
model = models_new[emp1.fold_index]

grid = {feature: grid_values}
df_modified = modify_grid(emp1, grid)
scores_new = model.predict_proba(df_modified.drop(exclude_columns, axis=1)).T[1]

plt.plot(grid_values, scores_new, "o-", label="predicted score")
plt.plot([emp1[feature]] * 2, [0, 1], "--", color="gray", label="current value")
plt.xlabel(feature)
plt.ylabel("Attrition score")
_ = plt.legend()

We can inspect both variables at once:

In [ ]:

grid = {
    "average_monthly_hours": np.linspace(100, 350, 51),
    "number_project": np.arange(1, 10),
}

df_modified = modify_grid(emp1, grid)
model = models_new[emp1.fold_index]
yscores = model.predict_proba(df_modified.drop(exclude_columns, axis=1)).T[1]

In [ ]:
sns.heatmap(
    df_modified.assign(attr_score=yscores)
    .sort_values("attr_score", ascending=False)
    .pivot(
        index="number_project", columns="average_monthly_hours", values="attr_score"
    ),
    cmap="coolwarm",
    vmin=0,
    vmax=1,
)

emp1[["number_project", "average_monthly_hours"]]

So according to our model a possible strategy would be to make the employee work on more projects even if some more working time is required...

**Note**: The evaluation time is really small, which allows us to test a **large number** of combinations in very little time.

In [ ]:
_ = sns.kdeplot(
    data=df_attr_new.assign(high_score=df_attr_new.attr_score > 0.8).query(
        "has_left == 0"
    ),
    x="number_project",
    y="average_monthly_hours",
    hue="high_score",
    common_norm=False,
)

So let's select an employee with a high number of projects:

In [ ]:
emp2 = (
    df_attr_new.query("has_left == 0 and number_project > 4")
    .sort_values("attr_score", ascending=False)
    .iloc[0]
)
emp2

**Exercise**: Reproduce the same analysis with the employee above.

The general idea would be to find the minimal change that allows us to pass below the 0.5 level of attrition probability.

Automatizing the optimal strategy search will imply to be able to associate a **cost** to the changes which is a complicated task in general. This is the kind of topic which is handled in the field of **operational research**:

$$ \min_{\Delta x, \Delta y} \textrm{Cost}(\Delta x, \Delta y)$$
$$ \text{s.t.   } z < 0.5 $$

## Assessing the performance

### Measuring the impact of a strategy

Imagine that we have designed a strategy to address employees. We want to deploy it and, of course measure the impact of this strategy.

**Exercise**: How would you proceed to measure the performance of the strategy?

### AB testing

AB testing usually consists in comparing 2 groups of people to which different strategies are used. It is quite common in e-commerce to test 2 versions of a web page, of an e-mailing campaign, etc. It is then possible to compare the **transformation rate** of each version.

The most common way to affect people to a group would be to do it **randomly**. There are usually many external factors which can have an influence on the output. If we have some prior knowledge on those factors, we should take them into account and affect people to groups so that the effect of those external factors is approximately the same in each group. This is called a **stratified** affectation.

AB testing would allow us to do two thing:
* Assess the model performance (do people with a high score actually leave more frequently than the others?)
* Assess the strategy.

One should keep in mind that using this kind of approach with HR can raise some **ethical issues**. If we do not want to keep some employees out of the campaign, it is possible to design two strategies (one using the model and the other one not using it) and compare their results.

Note that this testing protocol can be applied in all cases where we want to test the performance of a new strategy, and does not require using a ML model.

## Discussion on the applicability

### Issues with attrition case study ?

**Exercise**: Can you identify some issues (modeling, feature engineering, etc) in the way we performed the case study ?

## Identify potential application domains

We have presented 2 case studies which use the most 2 common supervised tasks in Machine Learning:

* Regression (predicting a continuous value)
* Classification (predicting classes)


**Exercise**: Can you cite other subjects for which those approaches could be used ?

### Special cases

* Activity forecast (regression)

    * Forecasting is a special case in which time is important. This has deep consequences on the data preparation and the validation strategies to use.
    * This falls in the domain of **time-series forecasting**

* Text classification (classification)
    * There are many classification tasks which imply text as input
    * Some specific techniques can be used to **vectorize** texts
    * This is related to the NLP (natural language processing) domain.

* Computer vision:
    * Processing images require some specific techniques
    * Some techniques are similar to those used in NLP

### ML as an application component

As we tried to show, a ML model is sometimes only a subpart of a more complete system. In the absence of business performance measure for the ML algorithm itself, it can be possible to assess the performance of a subpart of a system containing the model.

**Example**: In **credit card fraud detection**, we need to identify stolen cards. However, most often we will try to detect fraud at the transaction level. The transaction prediction will then need to be aggregated at the card level.

* The business performance (saved amount vs unnecessarily blocked cards) is computed at the card level
* The model performance is measured at the transaction level using a standard measure.

### Identify the steps of the project

**Exercise**: Can you sum-up the different steps of a ML project ?

**Exercise**: How would it apply to one of the previously cited topics ?